In [1]:
import pandas as pd
import numpy as np

In [ ]:

test = pd.read_csv('../input/ashrae-energy-prediction/test.csv', index_col=0, parse_dates = ['timestamp'])
building = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv', usecols=['site_id', 'building_id'])

test = test.merge(building, left_on = "building_id", right_on = "building_id", how = "left")
t = test[['building_id', 'meter', 'timestamp']]
t['row_id'] = t.index

#v1:ashrae-kfold-lightgbm-without-leak-1-08.ipynb
#v2:ashrae-simple-data-cleanup-lb-1-08-no-leaks.ipynb
#v3:another-1-08-lb-no-leak.ipynb
submission_base1 = pd.read_csv('../sub/submission_1.08_v1.csv', index_col=0)
submission_base2 = pd.read_csv('../sub/submission_1.08_v2.csv', index_col=0)
submission_base3 = pd.read_csv('../sub/submission_1.08_v3.csv', index_col=0)
submission_base4 = pd.read_csv('../sub/submission_1.10_v1.csv', index_col=0)

submission_sub = np.expm1(np.log1p(submission_base1)*0.3+
                          np.log1p(submission_base2)*0.3+
                          np.log1p(submission_base3)*0.3+
                          np.log1p(submission_base4)*0.1)
test["meter_reading"] = submission_sub.values
import pandas as pd
leak_df = pd.read_csv('../input/leakdata/leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')
test['meter_reading'] = [a if (a >= 0 and site in [0,1,2,4,15]) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

pub = pd.read_csv("../input/ashrae-energy-prediction/sample_submission.csv")
pub["meter_reading"] = test['meter_reading']
pub = pub.rename(columns = {"meter_reading":'meter_reading_pub'})



In [ ]:
# pub = pd.read_csv('../sub/combv6sub.csv').rename(columns = {"meter_reading":'meter_reading_pub'})
# lgb = pd.read_csv('../sub/lgb_delete_nulls_from_electrizity').rename(columns = {"meter_reading":'meter_reading_lgb'})
# xgb = pd.read_csv('../sub/xgb_leak').rename(columns = {"meter_reading":'meter_reading_xgb'})
lgb = pd.read_csv('../sub/lgb_3wise.csv').rename(columns = {"meter_reading":'meter_reading_lgb'})
xgb = pd.read_csv('../sub/xgb_3wise.csv').rename(columns = {"meter_reading":'meter_reading_xgb'})
cat = pd.read_csv('../sub/cb_3wise.csv').rename(columns = {"meter_reading":'meter_reading_cat'})
ffnn = pd.read_csv('../sub/ffnn.csv').rename(columns = {"meter_reading":'meter_reading_ffnn'})


test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')[["row_id", "building_id", "meter"]]
building = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv', usecols=['site_id', 'building_id'])
submit = test.merge(building, left_on = "building_id", right_on = "building_id", how = "left")
submit = submit.merge(lgb, on = "row_id", how =  "left")
submit = submit.merge(pub, on = "row_id", how =  "left")
submit = submit.merge(xgb, on = "row_id", how =  "left")
submit = submit.merge(cat, on = "row_id", how =  "left")
submit = submit.merge(ffnn, on = "row_id", how =  "left")






In [ ]:
submit['meter_reading'] = 0.3*submit['meter_reading_xgb']+0.3*submit['meter_reading_lgb']+\
                                    0.15*submit['meter_reading_cat']+0.25*0.9*submit['meter_reading_pub']

submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading'] = \
0.3*submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading_xgb'] +\
0.3*submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading_lgb'] +\
0.15*submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading_cat'] +\
0.20*0.9*submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading_pub'] +\
0.05*0.85*submit.loc[~submit.meter_reading_ffnn.isna(),'meter_reading_ffnn']




submit.loc[submit.site_id.isin([0,1,2,15]),'meter_reading'] = \
submit.loc[submit.site_id.isin([0,1,2,15]),'meter_reading_xgb']

submit.loc[submit.site_id.isin([4]),'meter_reading'] = \
0.5*submit.loc[submit.site_id.isin([4]),'meter_reading_lgb']+\
0.5*submit.loc[submit.site_id.isin([4]),'meter_reading_xgb']


submit.loc[(~submit.meter_reading_ffnn.isna()) & (submit.building_id==1099),'meter_reading'] = \
0.95 * 0.9 * submit.loc[(~submit.meter_reading_ffnn.isna()) & (submit.building_id==1099),'meter_reading_pub']+\
0.05 * 0.85 * submit.loc[(~submit.meter_reading_ffnn.isna()) & (submit.building_id==1099),'meter_reading_ffnn']

submit.loc[(submit.meter_reading_ffnn.isna()) & (submit.building_id==1099),'meter_reading'] = \
0.9 * submit.loc[(submit.meter_reading_ffnn.isna()) & (submit.building_id==1099),'meter_reading_pub']






In [ ]:
sub = pd.read_csv("../input/ashrae-energy-prediction/sample_submission.csv")

sub["meter_reading"] = 0.95*submit['meter_reading']
sub.to_csv("../sub/final.csv",  index = False,float_format="%.4f")

In [ ]:
sub.head()

In [ ]:
# !kaggle competitions submit -c ashrae-energy-prediction -f ../subsub/final_testv2.csv -m "final_testv2.csv"

